This workbook is a Python coding example utilizing the SAS QKB for data quality, enrichment, and entity resolution

In [13]:
# import swat (SAS Scripting Language for Analytics Transfer), 
#    and pandas       
import swat
import pandas as pd

In [14]:
# Create a connection to CAS, specifying host name or url  
# the SAS Viya server needs SAS Data Quality or SAS Data Preparation installed and QKB configured
viya=swat.CAS('http://99.999.99.99/cas-shared-default-http/', 5570, 'yourusername', 'yourpw', protocol='http')
# Set an active library for this session
viya.setsessopt(caslib='Public')

NOTE: 'Public' is now the active caslib.


+ Elapsed: 0.000494s, user: 0.00027s, sys: 0.000155s, mem: 0.274mb

In [15]:
# to upload the excel file uncomment the line below.  Not necessary if it's already uploaded, you can just use the exising table.
#testdata = viya.read_excel('C:\Testdata\CDN_ACCOUNTS.xlsx', casout=dict(name='CDN_Accounts',caslib='Public', promote='true'))

# use the swat CASTable object to treat a CAS Table like a pandas DataFrame
testdata = viya.CASTable('CDN_Accounts')
testdata.head(20)

NOTE: Cloud Analytic Services made the uploaded file available as table CDN_ACCOUNTS in caslib Public.
NOTE: The table CDN_ACCOUNTS has been created in caslib Public from binary data uploaded to Cloud Analytic Services.


,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,
1,202.0,Tony Sarducci,2125 31 Ave,Calgary,Alberta,T2T 1T5,100.0,,
2,203.0,Anthony Sarducci,,,,,400.0,tony.duke@telus.ca,
3,204.0,Amar Singh,5264 Joel Avenue,"Burlington,ON",L7L3Y7,,300.0,,
4,205.0,Jack Plant,,,,,50.0,555-2334,JPlante@gmail.com
5,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa ON K1Z 1E9,,613 755-2313,90.0,,ArnoldT@sas.com
6,207.0,Ms MJ Belanger,4500 Sherbrooke St W,Montreal,QC,H3Z 1E6,950.0,514 799 9239,
7,208.0,Mr. Anthony Sarducci,2125 31 Ave SW,Calgary AB T2T1T5,,,10.0,403.265.5177,tony.sarducci@bell.ca
8,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,,J8V3J5,150.0,555-2334,
9,210.0,Mme Marie-Josée Bélanger,"4500, rue Sherbrook Ouest",Montréal,PQ,,900.0,,MJBelanger@bell.ca


In [4]:
# run dataStep code invoking dq function to Identify contents of City, Province, PostCode fields
viya.dataStep.runCode( 
    code=''' data public.testexcel_dq_from_Python ;
             set public.CDN_Accounts ;
                 City_Ident = dqIdentify(City,'Field Content','ENCAN');
                 Prov_Ident = dqIdentify(Prov,'Field Content','ENCAN');
                 Post_Ident = dqIdentify(PostCode,'Field Content','ENCAN');
                 Phone_Ident = dqIdentify(Phone,'Field Content','ENCAN');
             run;''')
# let's look at just the first six rows of our data ... 
dq = viya.CASTable('testexcel_dq_from_Python')
dq.head(6)

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,City_Ident,Prov_Ident,Post_Ident,Phone_Ident
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,,CITY,STATE/PROVINCE,POSTAL CODE,PHONE
1,202.0,Tony Sarducci,2125 31 Ave,Calgary,Alberta,T2T 1T5,100.0,,,CITY,STATE/PROVINCE,POSTAL CODE,EMPTY
2,203.0,Anthony Sarducci,,,,,400.0,tony.duke@telus.ca,,EMPTY,EMPTY,EMPTY,E-MAIL
3,204.0,Amar Singh,5264 Joel Avenue,"Burlington,ON",L7L3Y7,,300.0,,,CITY-STATE/PROVINCE-POSTAL CODE,POSTAL CODE,EMPTY,EMPTY
4,205.0,Jack Plant,,,,,50.0,555-2334,JPlante@gmail.com,EMPTY,EMPTY,EMPTY,PHONE
5,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa ON K1Z 1E9,,613 755-2313,90.0,,ArnoldT@sas.com,CITY-STATE/PROVINCE-POSTAL CODE,EMPTY,PHONE,EMPTY


In [5]:
# using the above results of the SAS QKB Identity Analysis,
# run dataStep code to move stray phone, email, Postal code info to their correct places
viya.dataStep.runCode( 
    code=''' data public.testexcel_dq_from_Python;  
             set public.testexcel_dq_from_Python;
                 if Phone_Ident = 'E-MAIL' then do;
                       Email = Phone;
                       Phone = '';
                       Phone_Ident = 'EMPTY';
                    end;
                 if Post_Ident = 'PHONE' then do;
                       Phone = PostCode;
                       PostCode = '';
                       Phone_Ident = 'PHONE';
                       Post_Ident = 'EMPTY';
                    end;
                 if Prov_Ident = 'POSTAL CODE' then do;
                       PostCode = Prov;
                       Prov     = '';
                       Post_Ident = 'POSTAL CODE';
                       Prov_Ident = 'EMPTY';
                    end;
              run;''')
dq.head(6)

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,City_Ident,Prov_Ident,Post_Ident,Phone_Ident
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,,CITY,STATE/PROVINCE,POSTAL CODE,PHONE
1,202.0,Tony Sarducci,2125 31 Ave,Calgary,Alberta,T2T 1T5,100.0,,,CITY,STATE/PROVINCE,POSTAL CODE,EMPTY
2,203.0,Anthony Sarducci,,,,,400.0,,tony.duke@telus.ca,EMPTY,EMPTY,EMPTY,EMPTY
3,204.0,Amar Singh,5264 Joel Avenue,"Burlington,ON",,L7L3Y7,300.0,,,CITY-STATE/PROVINCE-POSTAL CODE,EMPTY,POSTAL CODE,EMPTY
4,205.0,Jack Plant,,,,,50.0,555-2334,JPlante@gmail.com,EMPTY,EMPTY,EMPTY,PHONE
5,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa ON K1Z 1E9,,,90.0,613 755-2313,ArnoldT@sas.com,CITY-STATE/PROVINCE-POSTAL CODE,EMPTY,EMPTY,PHONE


In [6]:
# run dataStep code to combine City, Province, PostCode fields for problem rows and parse out correct info
# using the SAS QKB parse definiton for City-State/Province-Postal Code.
viya.dataStep.runCode( 
    code=''' data public.testexcel_dq_2(drop=City_Ident Prov_Ident Post_Ident Phone_Ident parsedCPP);  
             set public.testexcel_dq_from_Python;
             if City_Ident ^= 'CITY' and (Prov_Ident='EMPTY' or Post_Ident='EMPTY') then do;
                 parsedCPP = dqParse(CATX(' ',City,Prov,PostCode), 'City - State/Province - Postal Code', 'ENCAN');
                 City     = dqParseTokenGet(parsedCPP, 'City', 'City - State/Province - Postal Code', 'ENCAN');
                 Prov     = dqParseTokenGet(parsedCPP, 'State/Province', 'City - State/Province - Postal Code', 'ENCAN');
                 PostCode = dqParseTokenGet(parsedCPP, 'Postal Code', 'City - State/Province - Postal Code', 'ENCAN');
                end;
             run;''')
dq2 = viya.CASTable('testexcel_dq_2')
dq2.to_frame()

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819)-555-2334,
1,202.0,Tony Sarducci,2125 31 Ave,Calgary,Alberta,T2T 1T5,100.0,,
2,203.0,Anthony Sarducci,,,,,400.0,,tony.duke@telus.ca
3,204.0,Amar Singh,5264 Joel Avenue,Burlington,ON,L7L3Y7,300.0,,
4,205.0,Jack Plant,,,,,50.0,555-2334,JPlante@gmail.com
5,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa,ON,K1Z 1E9,90.0,613 755-2313,ArnoldT@sas.com
6,207.0,Ms MJ Belanger,4500 Sherbrooke St W,Montreal,QC,H3Z 1E6,950.0,514 799 9239,
7,208.0,Mr. Anthony Sarducci,2125 31 Ave SW,Calgary,AB,T2T1T5,10.0,403.265.5177,tony.sarducci@bell.ca
8,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,,J8V3J5,150.0,555-2334,
9,210.0,Mme Marie-Josée Bélanger,"4500, rue Sherbrook Ouest",Montréal,PQ,,900.0,,MJBelanger@bell.ca


In [7]:
# use the SAS QKB Standardize defintions for Province, PostCode, and Phone to standardize those columns in place
# use the SAS QKB Gender Analysis definition to enrich the data with a gender field, based on the Name 
viya.dataStep.runCode( 
    code=''' data public.testexcel_dq_2;  
             set public.testexcel_dq_2;
             Prov    = dqStandardize(Prov,'State/Province (Postal Standard)','ENCAN');
             Phone   = dqStandardize(Phone,'Phone');
             PostCode= dqStandardize(PostCode,'Postal Code','ENCAN');
             gender  = dqGender(Name,'Name','ENCAN') ;
            run;''')
dq2.to_frame()

,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email,gender
0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819) 555 2334,,M
1,202.0,Tony Sarducci,2125 31 Ave,Calgary,AB,T2T 1T5,100.0,,,M
2,203.0,Anthony Sarducci,,,,,400.0,,tony.duke@telus.ca,M
3,204.0,Amar Singh,5264 Joel Avenue,Burlington,ON,L7L 3Y7,300.0,,,M
4,205.0,Jack Plant,,,,,50.0,555 2334,JPlante@gmail.com,M
5,206.0,Mr. Arnold Toporowski,"38 Metropol, Unit 1605",Ottawa,ON,K1Z 1E9,90.0,(613) 755 2313,ArnoldT@sas.com,M
6,207.0,Ms MJ Belanger,4500 Sherbrooke St W,Montreal,QC,H3Z 1E6,950.0,(514) 799 9239,,F
7,208.0,Mr. Anthony Sarducci,2125 31 Ave SW,Calgary,AB,T2T 1T5,10.0,(403) 265 5177,tony.sarducci@bell.ca,M
8,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,,J8V 3J5,150.0,555 2334,,M
9,210.0,Mme Marie-Josée Bélanger,"4500, rue Sherbrook Ouest",Montréal,QC,,900.0,,MJBelanger@bell.ca,F


In [8]:
# using the SAS QKB matchcode definitions create matchcodes on Name, Address, City.
viya.dataStep.runCode( 
    code=''' data public.testexcel_dq_3;  
             set public.testexcel_dq_2;
             Name_matchcode55 = dqMatch(Name,'Name',55,'ENCAN');
             Address_matchcode70 = dqMatch(Address,'Address',70,'ENCAN');
             City_matchcode85 = dqMatch(City,'City',85,'ENCAN');
             if length(Phone) > 7 then Phone7 = substr(Phone,length(Phone)-7,8);
            run;''')
# let's look at results for rows 7 to 10 ...
dq3 = viya.CASTable('testexcel_dq_3')
dq3[['ID','Name','Address','City','Name_matchcode55','Address_matchcode70','City_matchcode85','Phone7']].fetch(from_=7,to=10)

,ID,Name,Address,City,Name_matchcode55,Address_matchcode70,City_matchcode85,Phone7
6,207.0,Ms MJ Belanger,4500 Sherbrooke St W,Montreal,MWBF$$$$$$$$$$$B$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$S5042YMY3$$$$$...,BP~YW$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,799 9239
7,208.0,Mr. Anthony Sarducci,2125 31 Ave SW,Calgary,4Y8J$$$$$$$$$$$~$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$HZH9KZ$$$$$$$$...,3WFY$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,265 5177
8,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,NWB~$$$$$$$$$$$C$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$Z8P$$$$$$$$$$$...,F~P$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,555 2334
9,210.0,Mme Marie-Josée Bélanger,"4500, rue Sherbrook Ouest",Montréal,MWBF$$$$$$$$$$$B$$$$$$$$$$$,$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$S5042YMY3$$$$$...,BP~YW$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$,


In [9]:
#load Entity Resolution CAS action set 
viya.loadactionset(actionset="entityRes")


NOTE: Added action set 'entityRes'.


[actionset]

 'entityRes'

+ Elapsed: 0.00253s, user: 0.00248s, mem: 0.2mb

In [10]:
# use the entityres.match CAS action to match rows on: 
#  (Name & Address & Postal Code)  OR  (Name & City & Province)  OR  (Name & Phone)  OR  (Name & Email)
#
dq_clustered = viya.CASTable('test_Clustered', groupBy='CLUSTERID', replace=True)
viya.entityres.match(clusterid='CLUSTERID',  
        intable=dq3,
        matchrules=[{'rule':[{'columns':['Name_matchcode55','Address_matchcode70','Postcode']},]},
                    {'rule':[{'columns':['Name_matchcode55','City_matchcode85','Prov']}]},
                    {'rule':[{'columns':['Name_matchcode55','Phone7']}]},
                    {'rule':[{'columns':['Name_matchcode55','Email']}]}
                   ],
        outtable=dq_clustered) 

# display cluster results.  CLUSTERID is a 24-byte character string 
dq_clustered[['CLUSTERID','ID','Name','Address','City','Phone','Email']].sort_values('CLUSTERID').to_frame()

,CLUSTERID,ID,Name,Address,City,Phone,Email
0,AAAAAAAAAAAAAAAAAAAAAA==,205.0,Jack Plant,,,555 2334,JPlante@gmail.com
1,AAAAAAAAAAAAAAAAAAAAAA==,215.0,Jacques Plante,14 Denise Unit 201,Cantley,,
2,AAAAAAAAAAAAAAAAAAAAAA==,212.0,Jacques Plante,"14, Chemin Denis, app 201",Cantley,,
3,AAAAAAAAAAAAAAAAAAAAAA==,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,555 2334,
4,AAAAAAAAAAAAAAAAAAAAAA==,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,(819) 555 2334,
5,AAAAAAAAAAABAAAAAAAAAA==,203.0,Anthony Sarducci,,,,tony.duke@telus.ca
6,AAAAAAAAAAABAAAAAAAAAA==,208.0,Mr. Anthony Sarducci,2125 31 Ave SW,Calgary,(403) 265 5177,tony.sarducci@bell.ca
7,AAAAAAAAAAABAAAAAAAAAA==,220.0,Tony Sarducci,2125 31 Av,Calgary,,tony.duke@telus.ca
8,AAAAAAAAAAABAAAAAAAAAA==,202.0,Tony Sarducci,2125 31 Ave,Calgary,,
9,AAAAAAAAAAADAAAAAAAAAA==,213.0,Amar Singh,5264 Joel Av,Burlington,(905) 637 5119,amar.singh@lost.com


In [11]:
# to make the clusters easier to see, use simple.groupByInfo CAS action to genereate numeric GroupID from alphanumeric CLUSTERID

dq_clust_nums = viya.CASTable('test_Clust_nums',replace=True)
dq_clustered.simple.groupByInfo(generatedColumns='GROUPID',
                                casout=dq_clust_nums,
                                includeDuplicates=True)
dq_clust_nums[['_GroupID_','ID','Name','Address','City','Prov','PostCode','Amount','Phone','Email']].sort_values('_GroupID_').to_frame()

,_GroupID_,ID,Name,Address,City,Prov,PostCode,Amount,Phone,Email
0,1.0,205.0,Jack Plant,,,,,50.0,555 2334,JPlante@gmail.com
1,1.0,215.0,Jacques Plante,14 Denise Unit 201,Cantley,QC,J8V 3J5,10.0,,
2,1.0,212.0,Jacques Plante,"14, Chemin Denis, app 201",Cantley,QC,,40.0,,
3,1.0,209.0,"Plant, Jack",201-14 Denis Rd,Gatineau,,J8V 3J5,150.0,555 2334,
4,1.0,201.0,Mr. Jacques Plante,14 Denis Road,Cantley,QC,J8V 3J5,50.0,(819) 555 2334,
5,2.0,203.0,Anthony Sarducci,,,,,400.0,,tony.duke@telus.ca
6,2.0,208.0,Mr. Anthony Sarducci,2125 31 Ave SW,Calgary,AB,T2T 1T5,10.0,(403) 265 5177,tony.sarducci@bell.ca
7,2.0,220.0,Tony Sarducci,2125 31 Av,Calgary,AB,,400.0,,tony.duke@telus.ca
8,2.0,202.0,Tony Sarducci,2125 31 Ave,Calgary,AB,T2T 1T5,100.0,,
9,3.0,213.0,Amar Singh,5264 Joel Av,Burlington,ON,,100.0,(905) 637 5119,amar.singh@lost.com


In [12]:
# run datastep code to create golden record for each cluster with best info, and total amount
viya.dataStep.runCode( 
    code=''' data public.test_done
                     (drop= max_Name min_gender max_Address max_City max_Postal tot_Amount max_Phone max_Email
                            CLUSTERID ID Name_matchcode55 Address_matchcode70 City_matchcode85 Phone7); 
             set public.test_Clust_Nums;
             by _GroupID_;
             
             retain   max_Name  min_gender  max_Address  max_City  max_Postal  tot_Amount  max_Phone  max_Email;
             if first._GroupID_ then do;
                      max_Name = Name;
                      min_gender = gender;
                      max_Address = Address;
                      max_City = City;
                      max_Postal = PostCode;
                      tot_Amount = Amount;
                      max_Phone = Phone;
                      max_Email = Email;
                      end;
                 else do;
                       if length(Name) < length(max_Name)       then Name = max_Name;       else max_Name = Name;
                       if gender > min_gender                   then gender = min_gender;   else min_gender = gender; 
                       if length(Address) < length(max_Address) then Address = max_Address; else max_Address = Address;
                       if length(City) < length(max_City)       then City = max_City;       else max_City = City;
                       if length(PostCode) < length(max_Postal) then PostCode = max_Postal; else max_Postal = PostCode;
                       tot_Amount = Amount + tot_Amount;
                       Amount = tot_Amount;
                       if length(Phone) < length(max_Phone)     then Phone = max_Phone;     else max_Phone = Phone;
                       if length(Email) < length(max_Email)     then Email = max_Email;     else max_Email = Email;
                       end;
             if last._GroupID_ then output;
    run;''')
dq_done = viya.CASTable('test_done')
dq_done[['_GroupID_','Name','gender','Address','City','Prov','PostCode','Amount','Phone','Email']].sort_values('_GROUPID_').head(20)


,_GroupID_,Name,gender,Address,City,Prov,PostCode,Amount,Phone,Email
0,1.0,Mr. Jacques Plante,M,"14, Chemin Denis, app 201",Gatineau,QC,J8V 3J5,300.0,(819) 555 2334,JPlante@gmail.com
1,2.0,Mr. Anthony Sarducci,M,2125 31 Ave SW,Calgary,AB,T2T 1T5,910.0,(403) 265 5177,tony.sarducci@bell.ca
2,3.0,Amar Singh,M,5264 Joel Avenue,Burlington,ON,L7L 3Y7,400.0,(905) 637 5119,amar.singh@lost.com
3,4.0,Mr. Arnold Toporowski,M,"38 Metropole Private, Unit 1605",Ottawa,ON,K1Z 1E9,590.0,(613) 755 2313,ArnoldT@sas.com
4,5.0,Mme Marie-Josée Bélanger,F,"4500, rue Sherbrook Ouest",Montréal,QC,H3Z 1E6,1950.0,(514) 799 9239,MJBelanger@bell.ca
5,6.0,Jean-Francois Tremblay,M,P.O. Box 123,St-Marc-du-Lac-Long,QC,G0L 1T0,340.0,(819) 555 4545,JFTremblay@bell.ca
